In [1]:
import numpy as np

DATASET_PATH = "dataset/riichi_ds_v02/0003000.npz"

loads = np.load(DATASET_PATH)
# x, y = torch.tensor(loads["x"].astype(np.float32)), torch.tensor(loads["y"].astype(np.float32))

In [8]:
print(loads.keys())

KeysView(NpzFile 'dataset/riichi_ds_v02/0003000.npz' with keys: x_hand, x_hand_red, x_river, x_river_red, x_river_riichi...)


In [12]:
varnames = ("x_hand", "x_hand_red", "x_river", 
"x_river_red", "x_river_riichi", "x_meld", 
"x_meld_red5", "x_meld_throw", "x_meld_throw_red", 
"x_dora", "x_score", "x_pool", "x_winds", "y")

for varname in varnames:
    print(varname)
    print(loads[varname].shape)

x_hand
(799885, 4, 4, 9)
x_hand_red
(799885, 3)
x_river
(799885, 4, 4, 24, 9)
x_river_red
(799885, 4, 24)
x_river_riichi
(799885, 4, 24)
x_meld
(799885, 4, 4, 4, 9)
x_meld_red5
(799885, 4, 3)
x_meld_throw
(799885, 4, 4, 4, 9)
x_meld_throw_red
(799885, 4, 3)
x_dora
(799885, 4, 4, 9)
x_score
(799885, 4)
x_pool
(799885,)
x_winds
(799885, 2, 9)
y
(799885, 4)


In [20]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import glob

class MahjongDataset(Dataset):
    def __init__(self, data_paths):
        self.varnames_x = ("x_hand", "x_hand_red", "x_river", 
        "x_river_red", "x_river_riichi", "x_meld", 
        "x_meld_red5", "x_meld_throw", "x_meld_throw_red", 
        "x_dora", "x_score", "x_pool", "x_winds")
        self.loads = [np.load(path) for path in data_paths]
        print(data_paths)

# train_ds = MahjongDataset(sorted(glob.glob("dataset/riichi_ds_v02/*.npz"))[:10])
test_ds = MahjongDataset(sorted(glob.glob("dataset/riichi_ds_v02/*.npz"))[10:])

['dataset/riichi_ds_v02/0033000.npz', 'dataset/riichi_ds_v02/0035544.npz']


In [21]:
test_ds.loads

[NpzFile 'dataset/riichi_ds_v02/0033000.npz' with keys: x_hand, x_hand_red, x_river, x_river_red, x_river_riichi...,
 NpzFile 'dataset/riichi_ds_v02/0035544.npz' with keys: x_hand, x_hand_red, x_river, x_river_red, x_river_riichi...]